Mishael Derla, FAU Erlangen-Nürnberg, winter term 2024/2025

In [1]:
from os import listdir

import numpy

import matplotlib as mpl
import matplotlib.pyplot as plt

from xyz_file_io import XYZ
from physics import pbc_iterator

We will decided against using a Voronoi-nearest neighbor notion and instead opted for
$$a_i:=\sum_{k\ne i}w(|\vec{x}_k-\vec{x}_i|)\cdot\left(\frac{\vec{F}^\text{(self)}_i}{F^\text{(self)}}\right)\cdot\left(\frac{\vec{F}^\text{(self)}_k}{F^\text{(self)}}\right)$$
with
$$w(r)=N\cdot\exp\left(-\frac{1}{2}\left(\frac{r}{\sigma}\right)^2\right)$$
where $\sigma$ is the particle diameter and $N$ a constant that one may choose to one's pleasing: if for example one would like particles in direct contact to be weighted with a $1$, one may choose $N$ by solving $w(\sigma)=1$, resulting in $N=\sqrt{e}$; we made the latter choice in all following figures.

In [2]:
def alignment(
		positions: numpy.ndarray,
		self_propulsions: numpy.ndarray,
		particle_diameter:float,
		box_sidelength:float,
		normalization:float=numpy.sqrt(numpy.e)
		) -> numpy.ndarray:
	'''
	Computes the degree of alignment of all particles with
	their environment, where, generally speaking positive
	values mean net anti alignment, 0 means no net alignment
	and negative values mean net anti-alignment. At equal
	distance, the alignment value grows proportionally to 
	the number of particles, where aligning particles in
	direct contact would contribute exatcly +1 alignment
		
	Args:
		positions:
			numpy.ndarray of shape (N,d) containing the
			positions of N particles in d spatial dimensions
		self_propulsions:
			numpy.ndarray of shape (N,d) containint the
			components of the self propulsion force of
			N particles in d spatial dimensions
		particle_diameter:
			diameter of a particle
		box_sidelegth:
			domain sidelength, the distance traveled before
			wrapping around in periodic boundary conditions
		normalization:
			proportionality factor for the weight funciton
			~ exp(-(1/2)*(r/sigma)**2) where sigma is particle
			diameter. Default value is sqrt(e)
			
	Returns.
		numpy.ndarray of shape (N), where
		(N,d) is the shape of positions, where the i-th
		entry contains the degree of alignment of
		particle i with its environment
	'''

	particle_number, spatial_dimensions = positions.shape
		
	alignment = numpy.zeros(particle_number)

	# normalized self-propulsions
	normalized_self_propulsions = numpy.array([
		propulsion / numpy.linalg.norm(propulsion)
		for propulsion in self_propulsions
	])
		
	# looping over all unique ...
	for i in range(particle_number):
		# ... pairs of particles ...
		for j in range(i):

			# for all periodic images
			for pbc_image in pbc_iterator(spatial_dimensions):

				# distance of i and j squared, including periodic images
				separation_ij = positions[j] - positions[i] + box_sidelength * numpy.array(pbc_image)

				# prevent expensive operations by making a ...
                # ... cheaper check; after 3 * sigma, the ...
				# ... curve is vansishingly small
				if any(separation_ij > 3.5 * particle_diameter):
					continue
				
				r_ij_sq = numpy.dot(separation_ij, separation_ij)

				# weight
				w = normalization * numpy.exp(-(1/2) * r_ij_sq / particle_diameter**2)

				# alignment contribution of j to i is ...
				# ... the same number as alignment ...
				# ... contribution of i to j
				alignment_ij = w * numpy.dot(
					normalized_self_propulsions[i],
					normalized_self_propulsions[j]
				)

				alignment[i] += alignment_ij
				alignment[j] += alignment_ij
	
	return alignment

In [ ]:
data_directory = 'first_alignment_study/phi=0.1_data_two_force'

xyz_files = {
	# simulation index : file path
	int(file_name.split('_')[2]) : f'{data_directory}/{file_name}'
	for file_name in listdir(data_directory)
	if file_name.endswith('.xyz')
}

alignment_histories = []

for i, path in xyz_files.items():

	print(f'({i}) Analyzing {path}')
		
	# current alignment history
	alignment_history_i = []
	metadata = None

	for block in XYZ.file_iterator(path):
		
		if metadata is None:
			# unpacking metadata hidden in comment(s), but doing so only once
			metadata = {
				statement.split('=')[0] : float(statement.split('=')[1])
				for statement in block.comment.split(';')[1].replace(' ', '').split(',')
			}

		box_sidelength = metadata['box_sidelength']
		particle_diameter = metadata['particle_diameter']

		# unpacking the xyz_data
		positions = block.xyz_data[:,(0,1)]
		self_propulsions = block.xyz_data[:,(2,3)]

		# append to current alignment
		alignment_history_i.append(
			numpy.mean(
				alignment(
					positions, self_propulsions,
					particle_diameter, box_sidelength
				)
			)
		)
	
	# after that, append to the array of alignment histories to ...
	# ... be averaged later
	alignment_histories.append(alignment_history_i)

In [ ]:
alignment_history_mean = numpy.mean(alignment_histories, axis=0)
alignment_history_std  = numpy.std(alignment_histories, axis=0)

dt = metadata['time_step']
step_count = len(alignment_history_mean)
times = dt * numpy.arange(step_count)

plt.plot(times, alignment_history_mean, color='black')
# for alignment_history in alignment_histories:
# 	plt.plot(times, alignment_history, color='black', alpha=0.1)
plt.fill_between(
    times,
    alignment_history_mean - alignment_history_std,
    y2=alignment_history_mean+alignment_history_std,
    color='black',
    alpha=0.4
)

# for alignment_history in alignment_histories:
#     plt.plot(times, alignment_history)#, color='black', alpha=0.2)

plt.axhline(y=0, color='black', linestyle='dotted')

plt.xlabel('time $t$')
plt.ylabel('mean local alignment')

plt.title(r'average alignment over time of fifteen simulations at $\phi=0.7$')

plt.savefig('alignment_increase_two_force.pdf')